In [1]:
using LinearAlgebra

S(x, A) = dot(x, A, x)
S(x) = S(x, A)

niters = 10^6
n = 10
A = randn(n, n)
x = randn(n);

In [2]:
@code_warntype S(x, A)

Variables
  #self#::Core.Const(S)
  x::Vector{Float64}
  A::Matrix{Float64}

Body::Float64
1 ─ %1 = Main.dot(x, A, x)::Float64
└──      return %1


In [3]:
@code_warntype S(x)

Variables
  #self#::Core.Const(S)
  x::Vector{Float64}

Body::Any
1 ─ %1 = Main.S(x, Main.A)::Any
└──      return %1


In [4]:
function f(niters, x, A)
    y = Vector{eltype(x)}(undef, niters)
    for i in 1:niters
        y[i] = S(x, A)
    end
    y
end

@code_warntype f(niters, x, A)

Variables
  #self#::Core.Const(f)
  niters::Int64
  x::Vector{Float64}
  A::Matrix{Float64}
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  y::Vector{Float64}
  i::Int64

Body::Vector{Float64}
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│   %2  = Core.apply_type(Main.Vector, %1)::Core.Const(Vector{Float64})
│         (y = (%2)(Main.undef, niters))
│   %4  = (1:niters)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_5 = Base.iterate(%4))
│   %6  = (@_5 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_5::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = Main.S(x, A)::Float64
│         Base.setindex!(y, %12, i)
│         (@_5 = Base.iterate(%4, %11))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return y


In [5]:
function f(niters, x)
    y = Vector{eltype(x)}(undef, niters)
    for i in 1:niters
        y[i] = S(x)
    end
    y
end

@code_warntype f(niters, x)

Variables
  #self#::Core.Const(f)
  niters::Int64
  x::Vector{Float64}
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  y::Vector{Float64}
  i::Int64

Body::Vector{Float64}
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│   %2  = Core.apply_type(Main.Vector, %1)::Core.Const(Vector{Float64})
│         (y = (%2)(Main.undef, niters))
│   %4  = (1:niters)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = Main.S(x)::Any
│         Base.setindex!(y, %12, i)
│         (@_4 = Base.iterate(%4, %11))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return y


In [6]:
using BenchmarkTools

@btime f(niters, $x, $A)
@btime f(niters, $x);

  51.096 ms (2 allocations: 7.63 MiB)
  88.823 ms (1999491 allocations: 38.14 MiB)
